In [16]:
from pytrends.request import TrendReq
import pandas as pd
import json
import csv
import statistics

import numpy as np

In [12]:
train_df = pd.read_csv("../exported/hash_tag_train.csv", header = 0)
cv_df = pd.read_csv("../exported/hash_tag_cv.csv", header = 0)
test_df = pd.read_csv("../exported/hash_tag_test.csv", header = 0)

In [46]:
train_df

,news_id,source_tweet_id,cascade_size,hash_tags,d0,d1
0,politifact14667,1013440971034517504,2,NaN,2018-06-25,2018-07-01
1,politifact14664,920361276244508672,1,NaN,2017-10-11,2017-10-17
2,politifact14548,1021032222873333760,1,"economy,RealEstate",2018-07-16,2018-07-22
3,politifact14548,1021447823487717377,1,RegisterToVote,2018-07-17,2018-07-23
4,politifact13921,685818721856831488,1,NaN,2016-01-03,2016-01-09
...,...,...,...,...,...,...
93627,politifact14856,940954228050325504,1,NaN,2017-12-07,2017-12-13
93628,politifact15287,929339306463940610,1,NaN,2017-11-05,2017-11-11
93629,politifact14548,1019470337392734209,1,NaN,2018-07-12,2018-07-18
93630,politifact13949,996273827490414592,1,NaN,2018-05-09,2018-05-15


In [42]:
CATEGORY = 16 # NEWS

In [ ]:
# dfs = [train_df, cv_df, test_df]
dfs = [test_df]
pytrends = TrendReq(hl='en-US', tz=360)

for i, df in enumerate(dfs):
    
    print(f"dataset {i}")
    
    popularity_scores_avg = []
    popularity_scores_sum = []
    
    for idx, row in df.iterrows():
        
        if type(row['hash_tags']) == str:
            tags = row['hash_tags'].split(",")
            tag_avg_scores = []
            
            for j in range(0, len(tags), 5):
                
                end = len(tags) if ((j+5) >= len(tags)) else (j+5)
                partial_tags = tags[j:end]
                
                try:
                    pytrends.build_payload(
                        partial_tags, cat=CATEGORY,
                        timeframe = f"{row['d0']} {row['d1']}", geo='US', gprop='')

                    result = pytrends.interest_over_time()
                    
                    if not result.empty:
                    
                        for tag in partial_tags:
                            if tag in result
                            tag_avg_scores.append(result[tag].mean())
                    
                except:
                    print(f"partial tag: {partial_tags}")
                    print(f"error occured at: row {idx}")
                    raise
                    
            tag_avg_scores = np.array(tag_avg_scores)
            
            popularity_scores_avg.append(tag_avg_scores.mean())
            popularity_scores_sum.append(tag_avg_scores.sum())
            
        else:
            popularity_scores_avg.append(0)
            popularity_scores_sum.append(0)
            
    df['tag_pop_avg'] = popularity_scores_avg
    df['tag_pop_sum'] = popularity_scores_sum